In [1]:
!pip install torch torchvision torchaudio

In [2]:
!pip install datasets

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset,load_metric
import torch

/home/sanjan/anaconda3/envs/py3115/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
device

'cuda'

In [6]:
train_set = pd.read_csv("train - Copy.csv")

In [7]:
test_set = pd.read_csv("test - Copy.csv")

In [31]:
train_set.shape

(287113, 3)

In [9]:
train_set.isnull().sum()


id            0
article       0
highlights    0
dtype: int64

In [10]:
test_set.isna().sum()

id            0
article       0
highlights    0
dtype: int64

In [11]:
len(train_set)

287113

In [12]:
train_set.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [13]:
train_set["id"]

0         0001d1afc246a7964130f43ae940af6bc6c57f01
1         0002095e55fcbd3a2f366d9bf92a95433dc305ef
2         00027e965c8264c35cc1bc55556db388da82b07f
3         0002c17436637c4fe1837c935c04de47adb18e9a
4         0003ad6ef0c37534f80b55b4235108024b407f0b
                            ...                   
287108    fffdfb56fdf1a12d364562cc2b9b1d4de7481dee
287109    fffeecb8690b85de8c3faed80adbc7a978f9ae2a
287110    ffff5231e4c71544bc6c97015cdb16c60e42b3f4
287111    ffff924b14a8d82058b6c1c5368ff1113c1632af
287112    ffffd563a96104f5cf4493cfa701a65f31b06abf
Name: id, Length: 287113, dtype: object

In [14]:
train_set["highlights"]

0         Bishop John Folda, of North Dakota, is taking ...
1         Criminal complaint: Cop used his role to help ...
2         Craig Eccleston-Todd, 27, had drunk at least t...
3         Nina dos Santos says Europe must be ready to a...
4         Fleetwood top of League One after 2-0 win at S...
                                ...                        
287108    Chelsea Clinton said question of running for o...
287109    Vanilla Ice, 47 - real name Robert Van Winkle ...
287110    America's most lethal sniper made comment in i...
287111    A swarm of more than one million has crossed b...
287112    Other 2016 hopefuls maintain that Bush's annou...
Name: highlights, Length: 287113, dtype: object

In [15]:
train_set["article"][1]

'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as "The Milk Man," of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch. In one instance, the complaint alleges, Mata arranged to pay two assassins to kill rival drug dealers. The killers would pose as cops, pulling over their targets before shooting them, according to the complaint. "Ultimately, the (organization) decided not to move forward with the murder plot, but Mata still received a payment for setting up the meetings," federal prosecutors said in a statement. The complai

In [16]:
new_data = train_set.drop(columns=["id"])

In [17]:
new_test = test_set.drop(columns=["id"])

In [18]:
new_data.head()

,article,highlights
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [19]:
new_test.head()

,article,highlights
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [20]:
import nltk
from nltk.tokenize import word_tokenize as wt
from tqdm import tqdm
from nltk.tokenize import sent_tokenize

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sanjan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
new_data.head()

,article,highlights
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [23]:
import transformers
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
from transformers import pipeline


2024-03-24 11:46:17.491772: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 11:46:18.164009: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 11:46:18.164108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 11:46:18.268103: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 11:46:18.505020: I tensorflow/core/platform/cpu_feature_guar

In [24]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
text = "the extinguishment or modification of any rights accruing by virtue of any agreement, lease or licence for the purpose of searching for, or winning, any mineral or mineral oil, or the premature termination or cancellation of any such agreement, lease or licence, shall be deemed to be void on the ground that it is inconsistent with, or takes away or abridges any of the rights conferred by 1 [article 14 or article 19]: Provided that where such law is a law made by the Legislature of a State, the provisions of this article shall not apply thereto unless such law, having been reserved for the consideration of the President, has received his assent:] 2 [Provided further that where any law makes any provision for the acquisition by the State of any estate and where any land comprised therein is held by a person under his personal cultivation, it shall not be lawful for the State to acquire any portion of such land as is within the ceiling limit applicable to him under any law for the time being in force or any building or structure standing thereon or appurtenant thereto, unless the law relating to the acquisition of such land, building or structure, provides for payment of compensation at a rate which shall not be less than the market value thereof.] "

In [26]:
pipe = pipeline("summarization",model = model_ckpt)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
summary = pipe(text)

In [28]:
summary

[{'summary_text': 'extinguishment or modification of rights accruing by virtue of any agreement, lease or licence for the purpose of searching for, or winning, any mineral or mineral oil, or the premature termination or cancellation of any such agreement, lease or licence, shall be deemed to be void on the ground that it is inconsistent with, or takes away or abridges any of the rights conferred by 1 [article 14 or article 19]:<n>Provided that where such law is a law made by the Legislature of a State, the provisions of this article shall not apply thereto unless such law, having been reserved for the consideration of the President, has received'}]

In [29]:
print(summary[0]["summary_text"].replace("<n>","\n"))

extinguishment or modification of rights accruing by virtue of any agreement, lease or licence for the purpose of searching for, or winning, any mineral or mineral oil, or the premature termination or cancellation of any such agreement, lease or licence, shall be deemed to be void on the ground that it is inconsistent with, or takes away or abridges any of the rights conferred by 1 [article 14 or article 19]:
Provided that where such law is a law made by the Legislature of a State, the provisions of this article shall not apply thereto unless such law, having been reserved for the consideration of the President, has received
